In [ ]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Set device for PyTorch
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

def predict(model, features_scaled, window_size, ticker, scaler, stock_data, enhanced_data, num_months=60):
    model.eval()
    predictions = []
    last_window = features_scaled[-window_size:]
    last_price = float(stock_data['Close'].iloc[-1].iloc[0])
    last_features = enhanced_data.iloc[-1]
    
    # Start from today
    today = datetime.now()
    
    # Generate the next 60 months (5 years)
    prediction_dates = [today + timedelta(days=30 * i) for i in range(1, num_months + 1)]  # Every 30 days
    
    # Verify the dates are correct
    print("\nPredicting for the next months:")
    for date in prediction_dates:
        print(date.strftime('%Y-%m-%d'))
    
    with torch.no_grad():
        current_window = torch.FloatTensor(last_window).unsqueeze(0).to(device)
        current_price = last_price
        
        for i in range(len(prediction_dates)):
            # Predict percentage change
            pct_change = model(current_window).item()
            predicted_price = current_price * (1 + pct_change)
            predictions.append(predicted_price)
            
            # Update for next prediction
            current_price = predicted_price
            
            # Create new window by shifting
            new_window = current_window.clone()
            new_window[0, :-1, :] = new_window[0, 1:, :]
            
            # Update the last row with the new price
            new_features = last_features.copy()
            new_features['Close'] = predicted_price
            new_features['Open'] = predicted_price
            new_features['High'] = predicted_price
            new_features['Low'] = predicted_price
            
            # Scale the new features
            scaled_new_features = scaler.transform(new_features.values.reshape(1, -1))[0]
            new_window[0, -1, :] = torch.FloatTensor(scaled_new_features).to(device)
            current_window = new_window

    # Create DataFrame for predicted prices
    prediction_df = pd.DataFrame({
        'Date': prediction_dates,
        'Predicted Price': predictions,
        'Pct_Change': [(p/last_price - 1)*100 for p in predictions]
    })
    
    print("\nPredictions for the next months:")
    print(prediction_df.to_string(float_format=lambda x: '{:.2f}'.format(x)))
    
    # Plot the predictions
    plt.figure(figsize=(12, 6))
    plt.plot(stock_data.index[-30:], stock_data['Close'][-30:], label='Historical')
    plt.plot(prediction_df['Date'], prediction_df['Predicted Price'], label='Predicted', linestyle='--')
    plt.title(f'{ticker} Stock Price Prediction (Monthly for Next 5 Years)')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    return prediction_df


In [ ]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from utils.models.train import train
from utils.models.save import save
from utils.models.load import load
from utils.stocks.download_stock_data import download_stock_data
from utils.stocks.add_options_indicators import add_option_indicators
from utils.stocks.get_breakout_stocks import get_breakout_stocks
from utils.training.long_term_growth import long_term_growth

model_name = "stocks_10x_breakout_model"

# Try to load existing model first
model, scaler, last_data = load(model_name)

if model is None:
    print(f"Training new model.")
    # these tickers are used for training
    # they create the model baseline that we want to beat or match
    reported_breakout_stocks = get_breakout_stocks()

    model, sclare = long_term_growth(reported_breakout_stocks, model_name)
    # Verify we have a NEW model and scaler before proceeding
    if model is None or scaler is None:
        print("No valid model and scaler available. Cannot proceed with predictions.")
        exit()

# Verify we have a PRE_LOADED model and scaler before proceeding
if model is None or scaler is None:
    print("No valid model and scaler available. Cannot proceed with predictions.")
    exit()

# for analysis, these tickers use the model for prediction!
# this is where you set your stock symbols you want to predict on
tickers = ["LCID", "NIO"]

# Run the model for each ticker
for ticker in tickers:
    print(f"\nGenerating predictions for {ticker}")
    stock_data = download_stock_data(ticker, 
                                    datetime.now() - timedelta(days=30),
                                    datetime.now())
    enhanced_data = add_option_indicators(stock_data, ticker)

    print(f"Enhanced data: {enhanced_data}")
    
    # Define available features (make sure this matches your training features)
    available_features = [
        'Open', 'High', 'Low', 'Close', 'Volume',
        '1w_return', '2w_return', '1m_return', '20d_vol',
        'RSI', 'MACD', 'MACD_signal', 'MACD_hist',
        'BBL', 'BBM', 'BBU',
        'STOCH_RSI_K', 'STOCH_RSI_D',
        'above_200ma', 'above_50ma'
    ]

    print(f"here? data: {enhanced_data}")
    
    features = enhanced_data[available_features].values
    if features.size == 0:
        raise ValueError("No valid features extracted")
    features_scaled = scaler.transform(features)
    predictions = predict(model, features_scaled, 20, ticker, scaler, stock_data, enhanced_data, num_months=60)